In [1]:
import gc
import json
import pandas as pd
import numpy as np
import seaborn as sns

from tqdm import tqdm_notebook
from glob import glob

from sklearn.decomposition import TruncatedSVD, NMF
from utils import loadpkl

%matplotlib inline

/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [15]:
# load feathers
files = sorted(glob('../features/*.feather'))
df = pd.concat([pd.read_feather(f) for f in tqdm_notebook(files, mininterval=60)], axis=1)

# split train & test
train_df = df[df['click_mode'].notnull()]
test_df = df[df['click_mode'].isnull()]

/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/pandas/io/feather_format.py:124: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  nthreads=int_use_threads)
/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/pyarrow/pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [17]:
# load pkls
df = loadpkl('../features/plans.pkl')
queries = loadpkl('../features/queries.pkl')
profiles = loadpkl('../features/profiles.pkl')
queries_pred = loadpkl('../features/queries_pred.pkl')

In [3]:
# merge
df = pd.merge(df, queries, on='sid', how='left')
df = pd.merge(df, profiles, on='pid', how='left')

del queries, profiles
gc.collect()

train_df = df[df['click_mode'].notnull()]

In [20]:
profiles

,pid,profile_0,profile_1,profile_2,profile_3,profile_4,profile_5,profile_6,profile_7,profile_8,...,profile_nmf_10,profile_nmf_11,profile_nmf_12,profile_nmf_13,profile_nmf_14,profile_nmf_15,profile_nmf_16,profile_nmf_17,profile_nmf_18,profile_nmf_19
0,196356,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.000000,0.000000,0.000000,0.002154,0.000000,0.227141,0.582369,0.000000,0.000000,0.000000
1,204083,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.000000,0.000000,0.000000,0.000432,0.263290,0.253451,0.037184,0.031712,0.438799,0.000000
2,170667,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.000000,0.340126,0.000000,0.446981,0.000000,0.003279,0.036299,0.272778,0.449704,0.000000
3,115511,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.000000,0.000000,0.870193,0.000000,0.261206,0.000000,0.579168,0.000000,0.475017,0.025671
4,129719,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.000000,0.335413,0.019589,0.005031,0.000000,0.000000,0.067071,0.000000,0.000000,0.712688
5,174347,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.000000,0.344770,0.000000,0.000778,0.001011,0.016849,0.037546,0.309161,0.000000,0.000000
6,143618,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.000000,0.346305,0.000000,0.000000,0.000000,0.000000,0.035549,0.000000,0.000000,0.699967
7,116999,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.703690,0.014221,0.000000,0.000000,0.000000,0.006256,0.578823,0.010415,0.000000,0.695244
8,194535,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.000000,0.000000,0.000000,0.001160,0.000000,0.000000,0.037276,0.000000,0.486650,0.091366
9,125275,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.000000,0.391847,0.000000,0.001504,0.000846,0.000000,0.037794,0.000000,0.000000,0.696909


In [22]:
tmp = pd.merge(queries[['sid','pid','click_mode']],profiles['pid'], on='pid',how='left')

In [24]:
tmp = tmp[tmp.click_mode.notnull()]

In [35]:
tmp = pd.concat([tmp,pd.get_dummies(tmp.click_mode.astype(int))],axis=1)

In [38]:
tmp2 = tmp.groupby('pid').mean()[[i for i in range(0,12)]]
tmp2

,0,1,2,3,4,5,6,7,8,9,10,11
pid,,,,,,,,,,,,
100000.0,0.125000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.500000,0.125000,0.000000
100002.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
100005.0,0.066667,0.100000,0.333333,0.033333,0.000000,0.100000,0.000000,0.133333,0.000000,0.166667,0.033333,0.033333
100006.0,0.200000,0.000000,0.600000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
100007.0,0.250000,0.250000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000
100010.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000
100011.0,0.084211,0.132895,0.307895,0.047368,0.023684,0.081579,0.023684,0.153947,0.002632,0.094737,0.032895,0.014474
100012.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
100017.0,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [40]:
tmp2.idxmax(axis=1)

pid
100000.0    9
100002.0    1
100005.0    2
100006.0    2
100007.0    0
100010.0    7
100011.0    2
100012.0    1
100017.0    2
100018.0    7
100026.0    2
100030.0    4
100031.0    2
100033.0    2
100034.0    1
100036.0    7
100038.0    2
100043.0    7
100044.0    2
100045.0    2
100047.0    0
100048.0    2
100050.0    2
100051.0    0
100052.0    2
100053.0    0
100055.0    2
100057.0    1
100058.0    3
100059.0    9
           ..
216891.0    1
216893.0    1
216894.0    2
216895.0    7
216897.0    0
216898.0    2
216903.0    9
216910.0    7
216912.0    0
216918.0    4
216924.0    9
216926.0    1
216932.0    9
216935.0    7
216940.0    7
216943.0    2
216951.0    5
216953.0    1
216954.0    7
216959.0    9
216962.0    1
216967.0    2
216968.0    0
216969.0    2
216970.0    0
216972.0    0
216973.0    1
216975.0    7
216976.0    7
216977.0    4
Length: 46191, dtype: int64